In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model, svm
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
import sklearn.metrics as sklm
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [2]:
ave_month = pd.read_csv('AW_AveMonthSpend.csv')
bike_buyer = pd.read_csv('AW_BikeBuyer.csv')
work_cust = pd.read_csv('AdvWorksCusts.csv')

In [3]:
work_cust['BikeBuyer'] = bike_buyer['BikeBuyer']
work_cust['AveMonthSpend'] = ave_month['AveMonthSpend']
work_cust.head(5).transpose()

,0,1,2,3,4
CustomerID,11000,11001,11002,11003,11004
Title,NaN,NaN,NaN,NaN,NaN
FirstName,Jon,Eugene,Ruben,Christy,Elizabeth
MiddleName,V,L,NaN,NaN,NaN
LastName,Yang,Huang,Torres,Zhu,Johnson
Suffix,NaN,NaN,NaN,NaN,NaN
AddressLine1,3761 N. 14th St,2243 W St.,5844 Linden Land,1825 Village Pl.,7553 Harness Circle
AddressLine2,NaN,NaN,NaN,NaN,NaN
City,Rockhampton,Seaford,Hobart,North Ryde,Wollongong
StateProvinceName,Queensland,Victoria,Tasmania,New South Wales,New South Wales


In [4]:
work_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16519 entries, 0 to 16518
Data columns (total 25 columns):
CustomerID              16519 non-null int64
Title                   88 non-null object
FirstName               16519 non-null object
MiddleName              9534 non-null object
LastName                16519 non-null object
Suffix                  2 non-null object
AddressLine1            16519 non-null object
AddressLine2            276 non-null object
City                    16519 non-null object
StateProvinceName       16519 non-null object
CountryRegionName       16519 non-null object
PostalCode              16519 non-null object
PhoneNumber             16519 non-null object
BirthDate               16519 non-null object
Education               16519 non-null object
Occupation              16519 non-null object
Gender                  16519 non-null object
MaritalStatus           16519 non-null object
HomeOwnerFlag           16519 non-null int64
NumberCarsOwned         16519

In [5]:
cont_duplicates = work_cust.shape[0]
unique_df = work_cust.CustomerID.unique().shape[0]
print("Total:", cont_duplicates,
      "Unique:", unique_df)
print(cont_duplicates-unique_df)

Total: 16519 Unique: 16404
115


In [6]:
print(cont_duplicates)
work_cust.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(work_cust.shape[0])

16519
16404


In [7]:
work_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 0 to 16518
Data columns (total 25 columns):
CustomerID              16404 non-null int64
Title                   88 non-null object
FirstName               16404 non-null object
MiddleName              9453 non-null object
LastName                16404 non-null object
Suffix                  2 non-null object
AddressLine1            16404 non-null object
AddressLine2            273 non-null object
City                    16404 non-null object
StateProvinceName       16404 non-null object
CountryRegionName       16404 non-null object
PostalCode              16404 non-null object
PhoneNumber             16404 non-null object
BirthDate               16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404

In [8]:
# Drop columns with too many missing values
work_cust.drop(['Title','MiddleName','Suffix', 'AddressLine2'], axis = 1, inplace = True)

In [2]:
work_cust.to_csv('work_cust.csv')

NameError: name 'work_cust' is not defined

In [5]:
work_cust = pd.read_csv('work_cust.csv',index_col='CustomerID')
#work_cust.drop(['Unnamed: 0','AddressLine1','FirstName','LastName','StateProvinceName',
 #               'PhoneNumber','BirthDate','AveMonthSpend','PostalCode'], axis = 1, inplace = True)
#work_cust.head().transpose()

In [6]:
work_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16404 entries, 11000 to 18982
Data columns (total 13 columns):
City                    16404 non-null object
CountryRegionName       16404 non-null object
Education               16404 non-null object
Occupation              16404 non-null object
Gender                  16404 non-null object
MaritalStatus           16404 non-null object
HomeOwnerFlag           16404 non-null int64
NumberCarsOwned         16404 non-null int64
NumberChildrenAtHome    16404 non-null int64
TotalChildren           16404 non-null int64
YearlyIncome            16404 non-null int64
BikeBuyer               16404 non-null int64
Age                     16404 non-null int64
dtypes: int64(7), object(6)
memory usage: 1.8+ MB


In [18]:
labels = np.array(work_cust['BikeBuyer'])
labels.shape

(16404,)

In [19]:
# Encode categorical data
def encode_string(cat_features):
    
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education','Occupation',
                       'Gender','MaritalStatus','HomeOwnerFlag']

Features = encode_string(work_cust['CountryRegionName'])

for col in categorical_columns:
    temp = encode_string(work_cust[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)

(16404, 22)


In [20]:
# Add numerical feature
Features = np.concatenate([Features, np.array(work_cust[['NumberCarsOwned',
                                                        'NumberChildrenAtHome',
                                                        'TotalChildren',
                                                        'YearlyIncome','Age']])], axis = 1)

print(Features.shape)

(16404, 27)


In [21]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.3)
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = Features[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [22]:
scaler = preprocessing.StandardScaler().fit(X_train[:,23:])
X_train[:,23:] = scaler.transform(X_train[:,23:])
X_test[:,23:] = scaler.transform(X_test[:,23:])

In [23]:
#NB_mod = GaussianNB()
nn_mod = MLPClassifier(hidden_layer_sizes = (50,))
#logistic_mod = linear_model.LogisticRegression()
svm_mod = svm.LinearSVC()

nn_mod.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [25]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
print(logistic_mod.intercept_)
print(logistic_mod.coef_)

[-0.27751426]
[[-0.16678628  0.03838244  0.02008461 -0.03147848 -0.07420414 -0.06351242
   0.27128502  0.03396885 -0.12225425 -0.05259184 -0.40792205 -0.03038277
  -0.6129037   0.2068473   0.04220996  0.11671495 -0.469791    0.19227674
  -0.9121719   0.63465764 -0.20275495 -0.07475931 -0.09416452  1.08286555
   0.1054554   0.60853632 -0.43416441]]


In [28]:
probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:15,:])

[[0.68666082 0.31333918]
 [0.54120996 0.45879004]
 [0.60342194 0.39657806]
 [0.76780325 0.23219675]
 [0.9075406  0.0924594 ]
 [0.96346451 0.03653549]
 [0.92348004 0.07651996]
 [0.93178752 0.06821248]
 [0.94394833 0.05605167]
 [0.78656    0.21344   ]
 [0.98722476 0.01277524]
 [0.42219234 0.57780766]
 [0.76866468 0.23133532]
 [0.68674393 0.31325607]
 [0.82221836 0.17778164]]


In [29]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.5)
print(np.array(scores[:15]))
print(y_test[:15])

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[1 0 1 1 0 0 0 0 0 0 0 1 0 0 0]


In [30]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])


    
print_metrics(y_test, scores)

                 Confusion matrix
                 Score positive    Score negative
Actual positive      2955               349
Actual negative       710               908

Accuracy  0.78
 
           Positive      Negative
Num case     3304          1618
Precision    0.81          0.72
Recall       0.89          0.56
F1           0.85          0.63


In [31]:
ads_test = pd.read_csv('AW_test.csv')
ads_test.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


In [32]:
# Calculate age of customer based on birthdate
def calculate_age(end):
    r = relativedelta(pd.to_datetime('now'), pd.to_datetime(end)) 
    return '{}'.format(r.years)


ads_test['Age'] = ads_test['BirthDate'].apply(calculate_age)

# Convert age to integer
ads_test['Age'] = ads_test['Age'].astype('int64')

In [33]:
ads_totest = ads_test[['CountryRegionName','Education','Occupation','Gender',
                      'MaritalStatus','HomeOwnerFlag','NumberCarsOwned',
                      'NumberChildrenAtHome','TotalChildren','YearlyIncome',
                      'Age']]

ads_totest.head()

,CountryRegionName,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,Age
0,United States,Bachelors,Management,F,S,0,2,0,5,86931,74
1,Canada,Bachelors,Skilled Manual,M,M,1,2,2,4,100125,54
2,United States,Graduate Degree,Management,F,M,1,2,0,4,103985,85
3,United States,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161,60
4,France,High School,Manual,F,M,1,1,2,2,21876,53


In [34]:
# Encode categorical data
def encode_string(cat_features):
    
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education','Occupation',
                       'Gender','MaritalStatus','HomeOwnerFlag']

Features = encode_string(ads_totest['CountryRegionName'])

for col in categorical_columns:
    temp = encode_string(ads_totest[col])
    Features = np.concatenate([Features, temp], axis = 1)
    
print(Features.shape)

(500, 22)


In [35]:
# Add numerical feature
Features = np.concatenate([Features, np.array(ads_totest[['NumberCarsOwned',
                                                        'NumberChildrenAtHome',
                                                        'TotalChildren',
                                                        'YearlyIncome','Age']])], axis = 1)

print(Features.shape)

(500, 27)


In [36]:
X_test = Features
X_test[:,23:] = scaler.transform(X_test[:,23:])

In [37]:
scores = nn_mod.predict(X_test)
print(scores)

[0 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 1 1
 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0
 0 1 1 1 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1
 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0
 0 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1
 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1
 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 0 1
 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0
 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1
 0 1 0 0 0 1 0 0 0 1 1 0 

In [38]:
result = pd.DataFrame(scores, index = ads_test['CustomerID'],columns=['BikeBuyer'])

result
result.to_csv('resultclassification.csv')